In [ ]:
import datasets
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, DataCollatorWithPadding, Trainer
from transformers import AutoConfig

In [ ]:
data_files = {
    'train': 'data/pnli_train.csv',
    'validation': 'data/pnli_dev.csv',
}

In [ ]:
raw_datasets = datasets.load_dataset('csv', data_files=data_files, column_names=['sentence1', 'sentence2', 'labels'])

In [ ]:
# checkpoint = 'sentence-transformers/all-MiniLM-L12-v2'
# checkpoint_folder = 'sentence-transformers-all-MiniLM-L12-v2'

checkpoint = 'facebook/bart-large-mnli'
checkpoint_folder = 'facebook-bart-large-mnli'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    'checkpoints/' + checkpoint_folder + 'checkpoint',
    evaluation_strategy='steps',
    num_train_epochs=10,
    eval_steps=500,
    warmup_steps=200,
    metric_for_best_model='accuracy',
    load_best_model_at_end=True
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, ignore_mismatched_sizes=True)

In [ ]:
def compute_metrics(eval_preds):
    print(type(eval_preds))
    logits, labels = eval_preds
    logits = logits[0]
    predictions = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_datasets["validation"])

In [ ]:
trainer.save_model('models/' + checkpoint_folder)

In [ ]:
raw_test_datasets = datasets.load_dataset('csv', data_files={'test': 'data/pnli_test_unlabeled.csv'}, column_names=['sentence1', 'sentence2', 'labels'])

In [ ]:
tokenized_test_datasets = raw_test_datasets.map(tokenize_function, batched=True)

In [ ]:
tokenized_test_datasets['test'] = tokenized_test_datasets['test'].remove_columns(['sentence1', 'sentence2', 'labels'])

In [ ]:
tokenized_test_datasets.set_format(type='torch')

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
predictions, _, _ = trainer.predict(tokenized_test_datasets['test'])

In [ ]:
predictions = np.argmax(predictions, axis=1)

In [ ]:
len(predictions)